In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

# Read the Excel file (assumes the file is in the same directory as the notebook)
plane_df = pd.read_excel('sheeta.xlsx')

In [2]:
# Assume the Excel file has one row with columns: 'seat_total', 'seat_rows', 'seat_config'
seat_total = int(plane_df['seat_total'].iloc[0])
seat_rows = int(plane_df['seat_rows'].iloc[0])
seat_config = str(plane_df['seat_config'].iloc[0])

In [3]:
# Parse seat_config, e.g., "2-2" means 2 seats left, 2 seats right with an aisle in between
left_config, right_config = seat_config.split('-')
left_seats = int(left_config)
right_seats = int(right_config)
# Total columns: seats on left + aisle (1 column) + seats on right
total_columns = left_seats + right_seats + 1

In [4]:
# Create a visual representation of the plane as a 2D board (using a DataFrame for simplicity)
# We'll fill the seat cells with seat numbers (starting at 1) and mark the aisle with an "Aisle"
plane_board = []
seat_number = 1
for row in range(seat_rows):
    current_row = []
    # Left seats
    for _ in range(left_seats):
        if seat_number <= seat_total:
            current_row.append(str(seat_number))
            seat_number += 1
        else:
            current_row.append('')
    # Aisle column
    current_row.append("a")
    # Right seats
    for _ in range(right_seats):
        if seat_number <= seat_total:
            current_row.append(str(seat_number))
            seat_number += 1
        else:
            current_row.append('')
    plane_board.append(current_row)

In [5]:
# Create a DataFrame to display the plane layout visually
plane_layout_df = pd.DataFrame(plane_board)
print("Plane Environment Layout:")
print(plane_layout_df)

Plane Environment Layout:
      0    1  2    3    4
0     1    2  a    3    4
1     5    6  a    7    8
2     9   10  a   11   12
3    13   14  a   15   16
4    17   18  a   19   20
5    21   22  a   23   24
6    25   26  a   27   28
7    29   30  a   31   32
8    33   34  a   35   36
9    37   38  a   39   40
10   41   42  a   43   44
11   45   46  a   47   48
12   49   50  a   51   52
13   53   54  a   55   56
14   57   58  a   59   60
15   61   62  a   63   64
16   65   66  a   67   68
17   69   70  a   71   72
18   73   74  a   75   76
19   77   78  a   79   80
20   81   82  a   83   84
21   85   86  a   87   88
22   89   90  a   91   92
23   93   94  a   95   96
24   97   98  a   99  100
25  101  102  a  103  104
26  105  106  a  107  108
27  109  110  a  111  112
28  113  114  a  115  116
29  117  118  a  119  120


In [6]:
# Define a Passenger agent class with additional attributes
class Passenger:
    def __init__(self, seat_number, age, walking_speed, luggage, loading_speed, 
                 is_child=False, parent_seat=None, parent=False, num_children=0, late=False):
        self.seat_number = seat_number
        self.age = age
        self.walking_speed = walking_speed
        self.luggage = luggage
        self.loading_speed = loading_speed
        # Calculate loading time; avoid division by zero
        self.loading_time = luggage / loading_speed if loading_speed != 0 else float('inf')
        self.is_child = is_child      # True if this passenger is a child
        self.parent = parent          # True if this passenger is a parent (non-child)
        self.num_children = num_children  # Number of children if parent is True
        self.parent_seat = parent_seat    # For child passengers, record parent's seat
        self.late = late              # True if the passenger is late to boarding

    def __repr__(self):
        base = (f"Passenger(seat={self.seat_number}, age={self.age}, walking_speed={self.walking_speed}, "
                f"luggage={self.luggage}, loading_speed={self.loading_speed}, loading_time={self.loading_time:.2f}")
        extras = []
        if self.parent:
            extras.append(f"parent=True, num_children={self.num_children}")
        if self.is_child:
            extras.append(f"child of seat {self.parent_seat}")
        if self.late:
            extras.append("late=True")
        extra_str = ", ".join(extras)
        if extra_str:
            base += ", " + extra_str
        base += ")"
        return base

In [7]:
# Function to generate a weighted random walking speed based on age:
# For ages 18-59: weights favor higher speeds (speed value as weight)
# For ages 60-99: weights favor lower speeds (11-speed as weight)
def generate_walking_speed(age, is_child=False):
    speeds = list(range(1, 11))
    if is_child:
        # For children:
        if age <= 8:
            weights = [11 - s for s in speeds]  # favor lower speeds
        else:  # 9-17
            weights = speeds  # favor higher speeds
    else:
        # For adults:
        if age < 55:
            weights = speeds  # favor higher speeds
        else:
            weights = [11 - s for s in speeds]  # favor lower speeds
    return random.choices(speeds, weights=weights, k=1)[0]

def generate_loading_speed(age, is_child=False):
    speeds = list(range(1, 11))
    if is_child:
        # For children:
        if age <= 8:
            weights = [11 - s for s in speeds]  # favor lower loading speed
        else:  # 9-17
            weights = speeds  # favor higher loading speed
    else:
        # For adults:
        if age < 55:
            weights = speeds  # favor higher loading speeds
        else:
            weights = [11 - s for s in speeds]  # favor lower loading speeds
    return random.choices(speeds, weights=weights, k=1)[0]

In [8]:
def generate_age():
    # Generate age for adult passengers based on desired percentages
    categories = [
        ("18-54", 70, (18, 54)),
        ("55-69", 20, (55, 69)),
        ("70-84", 5, (70, 84)),
        ("85-99", 5, (85, 99))
    ]
    total_weight = sum(weight for (_, weight, _) in categories)
    r = random.uniform(0, total_weight)
    upto = 0
    for label, weight, (low, high) in categories:
        if upto + weight >= r:
            return random.randint(low, high)
        upto += weight
    return random.randint(18, 99)

def get_closest_seat(target, available):
    # Return the seat from 'available' that minimizes the absolute difference to target
    if not available:
        return None
    return min(available, key=lambda x: abs(x - target))

available_seats = list(range(1, seat_total + 1))
passengers = []

# Continue generating until there are no available seats
while available_seats:
    # Generate an adult passenger from a random available seat
    seat = random.choice(available_seats)
    available_seats.remove(seat)
    
    age = generate_age()
    ws = generate_walking_speed(age, is_child=False)
    ls = generate_loading_speed(age, is_child=False)
    luggage = random.randint(0, 5)
    # 5% chance that a non-child passenger is late to boarding
    late = (random.random() < 0.05)
    
    # Determine if this adult is a parent (15% chance now)
    is_parent = (random.random() < 0.15)
    num_children = 1 if is_parent else 0
    
    passenger = Passenger(seat, age, ws, luggage, ls, is_child=False, parent=False, 
                          num_children=num_children, late=late)
    passengers.append(passenger)
    
    # If this passenger is a parent, generate one child agent
    if is_parent:
        children_late = late  # parent's lateness carries over to the child
        child_seat = get_closest_seat(passenger.seat_number, available_seats)
        if child_seat is not None:
            available_seats.remove(child_seat)
            child_age = random.randint(0, 17)
            child_ws = generate_walking_speed(child_age, is_child=True)
            child_ls = generate_loading_speed(child_age, is_child=True)
            child_luggage = random.randint(0, 5)
            child_passenger = Passenger(child_seat, child_age, child_ws, child_luggage, child_ls, 
                                          is_child=True, parent_seat=passenger.seat_number, 
                                          parent=False, num_children=0, late=children_late)
            passengers.append(child_passenger)

In [12]:
# Print out all generated passengers with their attributes in a formatted output.
# For child agents, include the parent's seat number.
print("\nGenerated Agents:")
for p in sorted(passengers, key=lambda x: x.seat_number):
    late_str = "late: True" if p.late else "late: False"
    if p.is_child:
        print(f"Seat {p.seat_number} - age: {p.age}, walking_speed: {p.walking_speed}, luggage: {p.luggage}, loading_speed: {p.loading_speed}, loading_time: {p.loading_time:.2f}, {late_str} (Child of Seat {p.parent_seat})")
    else:
        print(f"Seat {p.seat_number} - age: {p.age}, walking_speed: {p.walking_speed}, luggage: {p.luggage}, loading_speed: {p.loading_speed}, loading_time: {p.loading_time:.2f}, {late_str}")


Generated Agents:
Seat 1 - age: 38, walking_speed: 9, luggage: 0, loading_speed: 2, loading_time: 0.00, late: False
Seat 2 - age: 11, walking_speed: 8, luggage: 3, loading_speed: 10, loading_time: 0.30, late: False (Child of Seat 3)
Seat 3 - age: 50, walking_speed: 9, luggage: 4, loading_speed: 9, loading_time: 0.44, late: False
Seat 4 - age: 89, walking_speed: 4, luggage: 1, loading_speed: 4, loading_time: 0.25, late: False
Seat 5 - age: 80, walking_speed: 4, luggage: 0, loading_speed: 2, loading_time: 0.00, late: False
Seat 6 - age: 42, walking_speed: 10, luggage: 0, loading_speed: 8, loading_time: 0.00, late: False
Seat 7 - age: 15, walking_speed: 7, luggage: 1, loading_speed: 8, loading_time: 0.12, late: False (Child of Seat 6)
Seat 8 - age: 18, walking_speed: 3, luggage: 4, loading_speed: 8, loading_time: 0.50, late: False
Seat 9 - age: 32, walking_speed: 7, luggage: 1, loading_speed: 3, loading_time: 0.33, late: False
Seat 10 - age: 40, walking_speed: 10, luggage: 5, loading_spe

In [11]:
# Define the age categories and count passengers in each (including children)
age_categories = {
    "0-17": (0, 17),
    "18-54": (18, 54),
    "55-69": (55, 69),
    "70-84": (70, 84),
    "85-99": (85, 99)
}

# Initialize counts for each category
age_counts = {category: 0 for category in age_categories}

total_passengers = len(passengers)

# Count passengers for each age category (both adults and children)
for p in passengers:
    for category, (low, high) in age_categories.items():
        if low <= p.age <= high:
            age_counts[category] += 1
            break

# Display the percentage of passengers for each category
print("Percentage of all passengers in each age category:")
for category, count in age_counts.items():
    percentage = (count / total_passengers) * 100 if total_passengers > 0 else 0
    print(f"{category}: {percentage:.2f}%")

# Create a list of seat numbers for passengers that are late
late_seats = [p.seat_number for p in passengers if p.late]

print("\nSeats with late passengers:")
print(late_seats)

Percentage of all passengers in each age category:
0-17: 11.67%
18-54: 60.00%
55-69: 15.83%
70-84: 5.83%
85-99: 6.67%

Seats with late passengers:
[59, 97, 57, 78]
